# Read in data

In [2]:
from datetime import timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import duckdb
import pyarrow

airports = [
    "KATL",
    "KCLT",
    "KDEN",
    "KDFW",
    "KJFK",
    "KMEM",
    "KMIA",
    "KORD",
    "KPHX",
    "KSEA",
]

In [3]:
DATA_DIRECTORY = Path("code execution development data")

## ETD

In [4]:
def read_etd(airport):
    etd = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_etd.csv.bz2",
        parse_dates=["departure_runway_estimated_time", "timestamp"],
        cache_dates = True
    )
    return etd

## MFS

In [5]:
def read_mfs(airport):
    etd = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_mfs.csv.bz2",
        engine = 'pyarrow'
    )
    return etd

## Lamp

In [6]:
def read_lamp(airport):
    lamp = pd.read_csv(
        DATA_DIRECTORY / airport / f"{airport}_lamp.csv.bz2",
        parse_dates=["timestamp", "forecast_timestamp"],
        dtype={"temperature": "int16", "wind_direction":"int16", "wind_gust":"int16", "cloud_ceiling":"float16", "visibility":"int16"},
        cache_dates = True
    )
    return lamp

In [7]:
import warnings
warnings.filterwarnings('ignore')

## Test: KDEN

In [8]:
df = pd.read_csv(
    'prescreened train labels/prescreened_train_labels_KDEN.csv.bz2', 
    parse_dates=["timestamp"],
    cache_dates = True
)
df.shape

(3247949, 4)

In [9]:
# join submission_format and etd
# declare variables (tables)
airport = "KDEN"
etd = read_etd(airport)

In [10]:
# perform join
query = """
SELECT df.gufi, df.timestamp, df.minutes_until_pushback, etd.departure_runway_estimated_time FROM df
LEFT JOIN (
    SELECT gufi, timestamp, departure_runway_estimated_time FROM etd
    ORDER BY timestamp DESC
)
etd ON df.gufi = etd.gufi
WHERE etd.timestamp <= df.timestamp
AND etd.timestamp > df.timestamp - INTERVAL 30 HOUR
"""

In [11]:
# save results to dataframe
etd_df = duckdb.sql(query).df()
etd_df.head()

,gufi,timestamp,minutes_until_pushback,departure_runway_estimated_time
0,UAL752.DEN.SAT.201220.0051.0007.TFM,2020-12-21 03:30:00,6,2020-12-21 02:01:00
1,SKW4650.DEN.CID.201221.0117.0007.TFM,2020-12-22 01:15:00,1,2020-12-22 01:38:00
2,SKW3802.DEN.SEA.201221.0121.0035.TFM,2020-12-22 01:15:00,14,2020-12-22 01:45:00
3,SKW5059.DEN.LBF.201221.0117.0022.TFM,2020-12-22 01:00:00,14,2020-12-22 01:38:00
4,SKW5739.DEN.DRO.201220.0257.0036.TFM,2020-12-21 02:45:00,5,2020-12-21 03:20:00


In [12]:
etd_df = etd_df.drop_duplicates(subset=['gufi', 'timestamp'])
etd_df.shape

(3247949, 4)

In [13]:
# create new column with etd in seconds
etd_df.timestamp = pd.to_datetime(etd_df.timestamp)
etd_df.departure_runway_estimated_time = pd.to_datetime(etd_df.departure_runway_estimated_time)
etd_df['seconds'] = (etd_df.departure_runway_estimated_time - etd_df.timestamp).dt.total_seconds()

In [14]:
etd_df.drop(columns='departure_runway_estimated_time', inplace=True)
etd_df.head()

,gufi,timestamp,minutes_until_pushback,seconds
0,UAL752.DEN.SAT.201220.0051.0007.TFM,2020-12-21 03:30:00,6,-5340.0
1,SKW4650.DEN.CID.201221.0117.0007.TFM,2020-12-22 01:15:00,1,1380.0
2,SKW3802.DEN.SEA.201221.0121.0035.TFM,2020-12-22 01:15:00,14,1800.0
3,SKW5059.DEN.LBF.201221.0117.0022.TFM,2020-12-22 01:00:00,14,2280.0
4,SKW5739.DEN.DRO.201220.0257.0036.TFM,2020-12-21 02:45:00,5,2100.0


In [15]:
# join msf data and etd_df
mfs = read_mfs(airport)

In [16]:
# perform join
query = """
SELECT 
    etd_df.gufi, 
    etd_df.timestamp, 
    etd_df.minutes_until_pushback, 
    etd_df.seconds, 
    mfs.aircraft_engine_class,
    mfs.aircraft_type,
    mfs.major_carrier,
    mfs.flight_type,
    mfs.isdeparture
FROM etd_df
LEFT JOIN mfs ON etd_df.gufi = mfs.gufi
"""

In [17]:
etd_df = duckdb.sql(query).df()
etd_df.shape

(3247949, 9)

In [18]:
# join lamp data and etd_df
lamp = read_lamp(airport)

In [29]:
def filter_lamp(current_time):
    # weather data
    valid_lamp = lamp.loc[(lamp.timestamp <= current_time) 
                    & (lamp.timestamp > valid_time) 
                    & (lamp.forecast_timestamp <= current_time) 
                    & (lamp.forecast_timestamp > valid_time)]
    try:
        return valid_lamp.iloc[-1]
    except:
        return None

In [ ]:
times = df.timestamp.unique()
for t in times:

    # insert weather data
    indices = etd_df[etd_df.timestamp == t].index
    current_time = pd.to_datetime(t)
    valid_time = current_time - pd.Timedelta(30, unit='hours')

    current_forecast = filter_lamp(current_time)
    if current_forecast is None:
        etd_df.loc[indices, 'temperature'] = None
        etd_df.loc[indices, 'wind_direction'] = None
        etd_df.loc[indices, 'wind_speed'] = None
        etd_df.loc[indices, 'wind_gust'] = None
        etd_df.loc[indices, 'cloud_ceiling'] = None
        etd_df.loc[indices, 'cloud'] = None
        etd_df.loc[indices, 'visibility'] = None
        etd_df.loc[indices, 'precip'] = None
        etd_df.loc[indices, 'lightning_prob'] = None      

    else:
        etd_df.loc[indices, 'temperature'] = current_forecast.temperature
        etd_df.loc[indices, 'wind_direction'] = current_forecast.wind_direction
        etd_df.loc[indices, 'wind_speed'] = filter_lamp(current_time).wind_speed
        etd_df.loc[indices, 'wind_gust'] = current_forecast.wind_gust
        etd_df.loc[indices, 'cloud_ceiling'] = current_forecast.cloud_ceiling
        etd_df.loc[indices, 'cloud'] = current_forecast.cloud
        etd_df.loc[indices, 'visibility'] = current_forecast.visibility
        etd_df.loc[indices, 'precip'] = current_forecast.precip
        etd_df.loc[indices, 'lightning_prob'] = current_forecast.lightning_prob

added weather data for 2022-04-05T14:00:00.000000000
added weather data for 2022-04-05T14:15:00.000000000
added weather data for 2022-04-05T14:30:00.000000000
added weather data for 2022-04-05T14:45:00.000000000
added weather data for 2022-04-05T15:00:00.000000000
added weather data for 2022-04-06T14:00:00.000000000
added weather data for 2022-04-06T14:15:00.000000000
added weather data for 2022-04-06T14:30:00.000000000
added weather data for 2022-04-06T14:45:00.000000000
added weather data for 2022-04-06T15:00:00.000000000
added weather data for 2022-04-07T14:00:00.000000000
added weather data for 2022-04-07T14:15:00.000000000
added weather data for 2022-04-07T14:30:00.000000000
added weather data for 2022-04-07T14:45:00.000000000
added weather data for 2022-04-07T15:00:00.000000000
added weather data for 2022-04-08T14:15:00.000000000
added weather data for 2022-04-08T14:30:00.000000000
added weather data for 2022-04-08T14:45:00.000000000
added weather data for 2022-04-08T15:00:00.000

In [ ]:
etd_head.shape

## Feature Engineering

In [ ]:
etd_df.isnull().sum()

In [ ]:
etd_df = pd.get_dummies(data=etd_df, columns=['aircraft_engine_class',
                                              'aircraft_type',
                                              'major_carrier',
                                              'flight_type', 
                                              'isdeparture'])
etd_df.head()

## Decision Tree Regressor

In [ ]:
from sklearn.model_selection import train_test_split

X = etd_df.drop(columns=['gufi', 'timestamp', 'minutes_until_pushback'])
y = etd_df.minutes_until_pushback
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
from sklearn.tree import (DecisionTreeRegressor, plot_tree)

dtr = DecisionTreeRegressor(criterion='friedman_mse', max_depth=5, min_samples_leaf=5)
dtr.fit(X_train, y_train)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(36, 6))

plot_tree(dtr, feature_names=X.columns, label=None, ax=ax, fontsize=9)
plt.plot()

In [ ]:
# labels
# mse
# samples
# value

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

y_pred = np.round(dtr.predict(X_test))
MAE = mae(y_test, y_pred)
MAE

## Submission

In [ ]:
submission = pd.read_csv('code execution development data/submission_format.csv', header='infer')
submission.shape

In [ ]:
predictions = np.round((dtr.predict(X)))
predictions.shape

In [ ]:
submission.minutes_until_pushback = predictions
submission.head()

In [ ]:
submission.to_csv('submission.csv', header='infer', index=False)